In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline

tfd = tfp.distributions
tfe = tf.contrib.eager
ed = tfp.edward2

tf.enable_eager_execution()

In [5]:
print( tfp.distributions.Normal(loc=0., scale=1.) )
print( tfp.edward2.Normal(loc=0., scale=1.) )

tfp.distributions.Normal("Normal/", batch_shape=(), event_shape=(), dtype=float32)
RandomVariable("-2.1965818", shape=(), dtype=float32, device=/job:localhost/replica:0/task:0/device:CPU:0)


In [7]:
print( tfp.edward2.Normal(loc=0, scale=1).distribution )
print( tfp.distributions.Normal(loc=0, scale=1) )

tfp.distributions.Normal("Normal/", batch_shape=(), event_shape=(), dtype=float32)
tfp.distributions.Normal("Normal/", batch_shape=(), event_shape=(), dtype=float32)


In [9]:
print( tfp.edward2.Normal(loc=0, scale=1) )
print( tfp.edward2.RandomVariable(tfp.distributions.Normal(loc=0, scale=1)) )

RandomVariable("-0.9519267", shape=(), dtype=float32, device=/job:localhost/replica:0/task:0/device:CPU:0)
RandomVariable("-0.79647803", shape=(), dtype=float32, device=/job:localhost/replica:0/task:0/device:CPU:0)


In [10]:
def logistic_regression(X):
    w = ed.Normal(loc=tf.zeros(X.spape[1]), scale=1, name="coeffs")
    b = ed.Normal(loc=0., scale=1., name="intercept")
    
    y = ed.Bernoulli(
        logits=tf.tensordot(X, w, [[1], [0]]) + b,
        name="outcomes"
    )
    return y

In [12]:
num_features = 10
X = tf.random_normal([100, num_features])
y  =logistic_regression(X)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'spape'

In [13]:
def logistic_regression_posterior(num_features):
    posterior_w = ed.MultivariateNormalTriL(
        loc=tf.get_variable("w_loc", [num_features]),
        scale_tril=tfp.trainable_distributions.tril_with_diag_softplus_and_shift(
            tf.get_variable("w_scale", [num_features * (num_features+1) / 2])),
        name="w_posterior"
    )
    posterior_b = ed.Normal(
        loc=tf.get_variable("b_loc", []),
        scale=tfp.trainable_distributions.softplus_and_shift(
            tf.get_variable("b_scale", [])),
        name="b_posterior"
    )
    return posterior_w, posterior_b


In [14]:
def set_prior_to_posterior_mean(f, *args, **kwargs):
    name = kwargs.get("name")
    if name == "w":
        return posterior_w.distribution.mean()
    elif name == 'b':
        return posterior_b.distribution.mean()
    return f(*args, **kwargs)


In [15]:
with ed.interception(set_prior_to_posterior_mean):
    predictions = logistic_regression(X)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'spape'